In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df_gastro = pd.read_csv("Daten/cleaned_data/gastro_filtered.csv")
df_weather = pd.read_csv("Daten/cleaned_data/377_produkt_klima_tag.csv")


In [ ]:
df_gastro.info()

In [ ]:
df_gastro

In [ ]:
selected_rows = df_gastro.loc[df_gastro['Gastro_Id'] == "WZ08-55", ["Gastro_Id", "Monat","Jahr"]]
selected_rows

In [ ]:
df_weather

In [ ]:
df_weather["TMK"].min()

In [ ]:
df_weather["TMK"].min()

In [ ]:
df_weather['Monat'] = pd.to_datetime(df_weather['MESS_DATUM']).dt.month
df_weather['Jahr'] = pd.to_datetime(df_weather['MESS_DATUM']).dt.year


In [ ]:
df_avg_temp_month_year = df_weather.groupby(['Jahr', 'Monat'])['TMK'].mean().reset_index()

In [ ]:
month_mapping = {
    'Januar': 1,
    'Februar': 2,
    'März': 3,
    'April': 4,
    'Mai': 5,
    'Juni': 6,
    'Juli': 7,
    'August': 8,
    'September': 9,
    'Oktober': 10,
    'November': 11,
    'Dezember': 12
}

# Convert the "Monat" column to numerical values using the mapping dictionary
df_gastro['Monat'] = df_gastro['Monat'].map(month_mapping)

In [ ]:
#Beispiel zur veranschaulichung
selected_rows = df_gastro.loc[df_gastro['Gastro_Id'] == "WZ08-55", ["Gastro_Id", "Monat","Jahr",]]
selected_rows

In [ ]:
df_avg_temp_month_year

In [ ]:
merged_df = pd.merge(df_gastro, df_avg_temp_month_year, on=['Monat', 'Jahr'])
merged_df

In [ ]:
pd.set_option('display.max_rows', 60)
selected_rows = merged_df.loc[df_gastro['Gastro_Id'] == "WZ08-55", ["Gastro_Id", "Monat","Jahr","TMK"]]
selected_rows

In [ ]:
plt.scatter(merged_df['TMK'], merged_df['Umsatz in millionen'])
plt.title('Korrelation zwischen Durchschnittstemperatur und Umsatz')
plt.xlabel('Durchschnittstemperatur (TMK)')
plt.ylabel('Umsatz in Millionen')
plt.show()

In [ ]:
grouped = merged_df.groupby('Gastro_Id')
merged_df[merged_df['Gastro_Id'] == 'WZ08-55']

In [ ]:
merged_df.drop(["Monatscode","Gastro_Description"], axis=1, inplace=True)
merged_df["Jahr_Monat"] = merged_df["Jahr"].astype(str) + "-" + merged_df["Monat"].astype(str)

merged_df

In [ ]:
min_value = merged_df["TMK"].min()
min_value

In [ ]:
max_value = merged_df["TMK"].max()
max_value

In [ ]:
selected_rows = merged_df.loc[df_gastro['Gastro_Id'] == "WZ08-55"]

selected_rows

In [ ]:
unique_gastro_ids = merged_df['Gastro_Id'].unique()

for gastro_id in unique_gastro_ids:
    selected_rows_gastro = merged_df[merged_df['Gastro_Id'] == gastro_id]
    
    plt.figure(figsize=(100, 6))  # Größe des Plots anpassen
    
    # Plot für den Umsatz
    plt.subplot(2, 1, 1)  # Erste von zwei Zeilen
    plt.plot(selected_rows_gastro['Jahr_Monat'], selected_rows_gastro['Umsatz in millionen'], marker='o', color='blue')
    plt.title(f'Umsatz über die Jahre mit Jahr-Monat für Gastro-ID {gastro_id}')
    plt.xlabel('Jahr-Monat')
    plt.ylabel('Umsatz in Millionen')
    plt.grid(True)
    plt.xticks(rotation=45)  # Rotation der x-Achsenbeschriftungen für bessere Lesbarkeit
    
    # Plot für die Temperatur
    plt.subplot(2, 1, 2)  # Zweite von zwei Zeilen
    plt.plot(selected_rows_gastro['Jahr_Monat'], selected_rows_gastro['TMK'], marker='o', color='red')
    plt.title(f'Temperatur über die Jahre mit Jahr-Monat für Gastro-ID {gastro_id}')
    plt.xlabel('Jahr-Monat')
    plt.ylabel('Temperatur')
    plt.grid(True)
    plt.xticks(rotation=45)  # Rotation der x-Achsenbeschriftungen für bessere Lesbarkeit
    
    plt.tight_layout()  # Für einen verbesserten Layout-Abstand zwischen den Subplots
    
    plt.show()


In [ ]:
correlation = merged_df['Umsatz in millionen'].corr(merged_df['TMK'])

print("Korrelationskoeffizient zwischen Temperatur und Jahresumsatz:", correlation)

In [ ]:
# Prüfen auf einen Zusammenhang mit zeitlicher Verzögerung
lags = range(1, 13)  # Mögliche Verzögerungen von 1 bis 12 Monaten
correlation_with_lags = []
for lag in lags:
    # Verzögerte Durchschnittstemperatur um `lag` Monate
    merged_df['TMK_delayed'] = merged_df['TMK'].shift(periods=lag)
    # Berechnen der Korrelation zwischen verzögerter Durchschnittstemperatur und Umsatz
    correlation_with_lags.append(merged_df['Umsatz in millionen'].corr(merged_df['TMK_delayed']))

# Ermitteln des Lags mit der höchsten Korrelation
max_correlation_index = np.argmax(correlation_with_lags)
lag_with_max_correlation = lags[max_correlation_index]
max_correlation = correlation_with_lags[max_correlation_index]

print("Maximale Korrelation mit Lag von", lag_with_max_correlation, "Monaten:", max_correlation)